# Denoising Autoencoder
This example will use the already published script: [Convolutional Autoencoder](https://www.kaggle.com/pankaj1234/convolutionalautoencoder)
The main idea behind this example is to demonstrate the powerful concept of image denosing by using application of AutoEncoders.

> A, section here will also cover the aspect that how Autoencoder can be used to detect Anomaly w.r.t. to computer vision (images). 

In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from skimage import io
import os
%matplotlib inline


# Import the Convolutional Autoencoder utility script
from convolutionalautoencoder import CNAutoEnc
from tensorflow.keras.optimizers import Adam
        

### Helper Functions<br/>Load and reshape the data

In [ ]:
from keras.preprocessing.image import load_img, img_to_array, array_to_img

def arrReshape(arr):
    arr = arr.astype("float32")/255.0
    return arr

def arrShape(arr):
    arr = (arr.astype("float32")*255.0).astype("uint8")
    return arr

def getImageArray(folderPath):
    for dirname, _, filenames in os.walk('/kaggle/input/denoise/denoising-dirty-documents/train/train'):
        images = np.ndarray(shape=(len(filenames), 320, 320, 1),dtype=np.float32)
        i=0
        for filename in filenames:
            # reference Tensorflow : https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/load_img
            img = img_to_array(load_img(os.path.join(dirname,filename),color_mode="grayscale",target_size=(320,320), interpolation="nearest"))
            img= img.reshape((320,320,1))
            images[i] = img
            i+=1
    return images


### Load the data

In [ ]:
trainNoise = arrReshape(getImageArray('/kaggle/input/denoise/denoising-dirty-documents/train/train'))
trainCleaned = arrReshape(getImageArray('/kaggle/input/denoise/denoising-dirty-documents/train_cleaned/train_cleaned'))
testNoise = arrReshape(getImageArray('/kaggle/input/denoise/denoising-dirty-documents/test/test'))

### Noise Image and Original Image 

In [ ]:
array_to_img(np.hstack([trainNoise[5],trainCleaned[5]]))

In [ ]:
EPOCH=50
batchsize=5
(encoder,decoder, autoencoder) = CNAutoEnc.create((320,320,1))
autoencoder.compile(loss="mse",optimizer=Adam(lr=5e-4) )

In [ ]:
history = autoencoder.fit(trainNoise,trainCleaned,epochs=EPOCH,batch_size=batchsize)

In [ ]:
decodedImages = autoencoder.predict(testNoise)

### Display the results after denoising
Left Image is decoded from the Noise and Right side image is the Original Image. 

In [ ]:
sampleshow = None
for i in range(5):
    if sampleshow is None:
        sampleshow = np.hstack([decodedImages[i],testNoise[i]])
    else:
        sampleshow = np.vstack([sampleshow,np.hstack([decodedImages[i],testNoise[i]])])

In [ ]:
array_to_img(sampleshow)

# Anomaly Detection
In this section we can look into how the Autoencoder can be used to detect the Anamoly for input images.</br> The overall idea behind this concept is to use compress the image to is latent dimension and then reconstruct the Image from there itself.</br> The difference in MSE of the orignalimage on which the Autoencoder is trained and the image with the anomaly will help decide if the image is and detected anomaly.
### Theory behind this concept
We would expect the autoencoder to do a really good job at reconstructing the images from the domain on which its trained upon, as that is exactly what the autoencoder was trained to do — and if we were to look at the MSE between the input image and the reconstructed image, we would find that it’s quite low.

But now instead of that - if we feed an image from the different domain to the autoencoder (anamoly) then thit will generate the output (reconstructed) which is no where near that image (because autoencoder is not trained on this domain of images) - Hence the MSE is going to be very high - thus the anamoly can be detected.

#### Sample Image for Anamoly detection
In this step I am extracting a sample image from the TrainNoise set and then by doiong random rotation a sample image is derived  - this image is treated as an anamoly for this original dataset. This is because this image is not trained on this network.

In [ ]:
from tensorflow.keras.preprocessing.image import random_rotation
origImage = trainNoise[0]
junkImage  = random_rotation(origImage, rg=20, row_axis=0, col_axis=12, channel_axis=2, fill_mode='nearest', cval=0.0, interpolation_order=1)
junkImage = arrReshape(junkImage.reshape((320, 320, 1)))

In [ ]:
array_to_img(junkImage)

#### Reconstruct the anamoly Image
In this step I am reconstructing the image from the autoencoder. By this reconstructed image we will see how far the autoencoder is able to reconstruct this image successfully; by looking into the MSE from the original anamoly (rotated) image.

In [ ]:
anamolyPredict = autoencoder.predict(junkImage.reshape((1,320, 320, 1)))

In [ ]:
mse = np.mean((origImage-anamolyPredict[0])**2)
print(mse)

### Anamoly Detection Output
Not to surprise the Autoencoder model is unable to construct the Anamoly image perfectly. Hence this is daignosed as an anamoly for the dataset on which the Autoancoder is trained originally.

In [ ]:
array_to_img(anamolyPredict[0])